In [1]:
from selenium import webdriver
import json
import time

In [2]:
driver = webdriver.Chrome()


In [3]:
#url = "https://sf-item.taobao.com/sf_item/592140469999.htm?spm=a213w.7398504.paiList.1.a1da26cdBOcTJJ"

#url = "https://sf-item.taobao.com/sf_item/591732235950.htm?spm=a213w.7398504.paiList.6.7ede26cdIZuymq"

sourceUrl = "https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.61.22f526cdj7uTkT&category=50025969" \
    "&auction_source=0&province=%CC%EC%BD%F2&sorder=2&st_param=-1&auction_start_seg=-1"

In [4]:
driver.get(sourceUrl)

In [ ]:
urlList = []

In [ ]:
for i in range(2,3):
    if (i > 1):
        driver.find_element_by_link_text(str(i)).click()
    else:
        pass
    
   # time.sleep(1)
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    jd = jsonData["data"]
    for i in range(len(jd)):
    #for i in range(5):
        num = jd[i]["id"]
        url = "#pai-item-"+str(num)+" .pic"
        urlList.append(url)
        driver.find_element_by_css_selector(url).click()
        time.sleep(1)
        handles = driver.window_handles
        #print(handles)
        driver.switch_to.window(handles[1])
        data = driver.find_element_by_class_name("bid-fail").text
        bidStatus = data[5:7]
        if(bidStatus !="结束"):
            print("流拍 or 失败")
            print(str(i) +":"+ data+driver.title)
            print("url"+driver.current_url)
        else:
            url2 = driver.current_url+"#J_Confirmation"
            driver.get(url2)
            time.sleep(1)
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            name = info[4:index]
            print("结束"+":"+name)
            print(str(i) +":"+ data+driver.title)
            print("url"+driver.current_url)

        driver.close()
        driver.switch_to.window(handles[0])


流拍 or 失败
0:本场拍卖已流拍 ！天津市河西区郁江道与九华山路交口玉水园1-2-501号房地产 - 司法拍卖 - 阿里拍卖
urlhttps://sf-item.taobao.com/sf_item/590875029532.htm?spm=a213w.7398504.paiList.1.220126cd707y8I
流拍 or 失败
1:本场拍卖已流拍 ！天津市河西区太湖路与澧水道交口东南侧尚城32-1110号房地产 - 司法拍卖 - 阿里拍卖
urlhttps://sf-item.taobao.com/sf_item/590656776073.htm?spm=a213w.7398504.paiList.2.220126cd707y8I
流拍 or 失败
2:本场变卖失败天津市东丽区顶秀欣园10-1-303号房地产一套 - 司法拍卖 - 阿里拍卖
urlhttps://sf-item.taobao.com/sf_item/587452483779.htm?spm=a213w.7398504.paiList.3.220126cd707y8I
结束:朱艳秋
3:本场拍卖已结束 ！天津市滨海新区中新天津生态城和旭路1060号季景园9-101室 - 司法拍卖 - 阿里拍卖
urlhttps://sf-item.taobao.com/sf_item/590871909472.htm?spm=a213w.7398504.paiList.4.220126cd707y8I#J_Confirmation
流拍 or 失败
4:本场拍卖已流拍 ！天津市北辰区引河南里1-4-302号房地产 - 司法拍卖 - 阿里拍卖
urlhttps://sf-item.taobao.com/sf_item/592296215593.htm?spm=a213w.7398504.paiList.5.220126cd707y8I
结束:郑学明
5:本场拍卖已结束 ！天津市蓟县碧玉家园北侧，福地花园南侧山水颐城北园4-1-201房地产 - 司法拍卖 - 阿里拍卖
urlhttps://sf-item.taobao.com/sf_item/592218695978.htm?spm=a213w.7398504.paiList.6.220126cd707y8I#J_Confirmation
流拍 or 失败

In [ ]:
type(data)

In [ ]:
#for i in range(len(urlList))


#     driver.switchTo().window("vars.get("root").toString()");
#     driver.findElement(By.cssSelector("#pai-item-591508706185 .pic")).click();
#     driver.switchTo().window("vars.get("win8140").toString()");
#     driver.close();

In [ ]:
#urlList

In [ ]:
#590871909472
#pai-item-591508706185 .pic

In [ ]:
#data = driver.find_element_by_class_name("bid-fail").text
#print(data)

In [ ]:
#d = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')


In [ ]:
#jd = json.loads(d)


In [ ]:
# for i in range(len(jd["data"])):
#     print(jd["data"][i]["id"])

In [ ]:
#driver.find_element_by_css_selector(url).click()